In [1]:
from pylab import plot,show
import numpy as np
import pandas as pd
from numpy import linalg as LA
import math
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve
from scipy.sparse.linalg import spsolve_triangular
from scipy.sparse import diags
plt.style.use('seaborn-white')

# Acoustic Wave Equation in Time Domain

The elastic wave equation can be stated as follows,
$$
\frac{\partial^2 u}{\partial t^2}=c^2 \nabla^2 u+f,\qquad x\in \Omega \subset \mathbb{R}^d,~t\in(T_1,T_2]
\qquad\qquad\qquad\qquad (1)
$$
where $\Delta$ is *the Laplacian*, $f$ is a *forcing function* (for example our source) and $c$ is *the wave velocity* at which the time and spatially varying wave $u$ propagates.

* [Finite-Difference Methods for the Three Dimensional Wave Equation](#Finite-Difference-Methods-for-the-Three-Dimensionals-Wave-Equation)
    * [Explicit Scheme](#Explicit-Scheme)
    * [Implicit Scheme](#Implicit-Scheme)
    * [A Fourth-order Scheme](#A-Fourth-order-Scheme)
* [Example 1](#Example-1)
    * [The Explicit Scheme](#The-Explicit-Scheme)
    * [The Implicit Scheme](#The-Implicit-Scheme)
* [Example 2](#Example-2)
    * [The Fourth-order Scheme](#The-Fourth-order-Scheme)

## The Finite-Difference Method for the three-dimensional wave equation

Let $I=(a,b)\times(c,d)\times(e,f)$ and $J=[a,b]\times[c,d]\times[e,f]$. A three-dimensional for of the wave equation presented in \eqref{eq1.02} can be found as follows,
\begin{align*}
\begin{cases}
\dfrac{\partial^2 u}{\partial t^2} = c^2(x,y,z) \Delta u +f(x,y,z,t),&(x,y,z)\in I,~t\in(T_1,T_2],\\
u(x,y,z,T_1)=g(x,y,z),&(x,y,z)\in J,\\
\dfrac{\partial }{\partial t}u(x,y,z,T_1) = s(x,y,z),&(x,y,z)\in J,\\
u(a,y,z,t) = f_{a}(y,z,t),&(y,z)\in [c,d]\times[e,f],~t \in[T_1,T_2],\\
u(b,y,z,t) = f_{b}(y,z,t),&(y,z)\in [c,d]\times[e,f],~t \in[T_1,T_2],\\
u(x,c,z,t) = f_{c}(x,z,t),&(x,z)\in [a,b]\times[e,f],~t \in[T_1,T_2],\\
u(x,d,z,t) = f_{d}(x,z,t),&(x,z)\in [a,b]\times[e,f],~t \in[T_1,T_2],\\
u(x,y,e,t) = f_{e}(x,y,t),&(x,y)\in [a,b]\times[c,d],~t \in[T_1,T_2],\\
u(x,y,f,t) = f_{f}(x,y,t),&(x,y)\in [a,b]\times[c,d],~t \in[T_1,T_2].
\end{cases}
\qquad \qquad (1)
\end{align*}

Consider the following notations:

* $h_x=\Delta x=\dfrac{b-a}{N_x},~h_y=\Delta y=\dfrac{d-c}{N_y},~h_z=\Delta z=\dfrac{f-e}{N_z}$ and $\tau=\Delta t=\dfrac{T_2-T_1}{N_t}$ where $N_x$, $N_y$, $N_z$ and $N_t$ are positive integers,
* $u_{i,j,k}^{n}= u(ih_x,jh_y,kh_z,n\tau)$ and $c_{i,j,k}=c(ih_x,jh_y,kh_z)$,
* $\lambda_x=\dfrac{\tau}{h_x}$, $\lambda_y=\dfrac{\tau}{h_y}$ and $\lambda_z=\dfrac{\tau}{h_z}$.

## Explicit Scheme
An explicit finite difference scheme for solving the problem (1) can be considered as follows,
\begin{align*}
\frac{1}{\tau^2} \delta_t^2 u_{i,j,k}^{n}&= c_{i,j,k}^2\left(\frac{1}{h_x^2} \delta_x^2+\frac{1}{h_y^2} \delta_y^2+\frac{1}{h_z^2} \delta_z^2\right)u_{i,j,k}^{n}+f_{i,j,k}^n.
\end{align*}
To develop a numerical algorithm, we can rewrite this as follows,
\begin{align*}
u_{i,j,k}^{n+1}&=c_{i,j,k}^2\left[\lambda_x^2\left(u_{i+1,j,k}^{n}+u_{i-1,j,k}^{n}\right)+\lambda_y^2\left(u_{i,j+1,k}^{n}+u_{i,j-1,k}^{n}\right)+\lambda_z^2\left(u_{i,j,k+1}^{n}+u_{i,j,k-1}^{n}\right)\right]
\notag \\ &
+2\left(1-c_{i,j,k}^2\left(\lambda_x^2+\lambda_y^2+\lambda_z^2\right)\right)u_{i,j,k}^{n}-u_{i,j,k}^{n-1}+\tau^2f_{i,j,k}^n,
\end{align*}

## Implicit Scheme
A general implicit finite difference form of the problem can be expressed as follows,
\begin{align*}
\frac{1}{\tau^2} \delta_t^2 u_{i,j,k}^{n}&= c_{i,j,k}^2 \left(\frac{1}{h_x^2} \delta_x^2+\frac{1}{h_y^2} \delta_y^2 +\frac{1}{h_z^2} \delta_z^2 \right)
\left[\alpha  u_{i,j,k}^{n+1}+\beta  u_{i,j,k}^{n} +\gamma u_{i,j,k}^{n-1}\right]+f_{i,j,k}^n,
\qquad \qquad (2)
\end{align*}
where $\alpha,~\beta$ and $\gamma$ are constants such that $\alpha+\beta+\gamma=1$.

The equation (2) can be expanded as follows,
\begin{align*}
\left(1-\alpha c^2_{i,j,k}\left(\lambda_x^2 \delta_x^2 +\lambda_y^2 \delta_y^2 +\lambda_z^2 \delta_z^2 \right)\right)u_{i,j,k}^{n+1}&=
\left[c_{i,j,k}^2 \beta \left(\lambda_x^2 \delta_x^2 +\lambda_y^2 \delta_y^2 +\lambda_z^2 \delta_z^2 \right)+2\right]u_{i,j,k}^{n}
\notag \\ &
 +\left[c_{i,j,k}^2\gamma \left(\lambda_x^2 \delta_x^2 +\lambda_y^2 \delta_y^2 +\lambda_z^2 \delta_z^2 \right)-1\right]u_{i,j,k}^{n-1}
 +\tau^2f_{i,j,k}^n.
\end{align*}
Note that,
\begin{align*}
&\left(1-\alpha c^2_{i,j,k}\left(\lambda_x^2 \delta_x^2 +\lambda_y^2 \delta_y^2 +\lambda_z^2 \delta_z^2 \right)\right)u_{i,j,k}^{n}
-\left(1-\alpha c^2_{i,j,k} \lambda_x^2 \delta_x^2\right)\left(1-\alpha c^2_{i,j,k} \lambda_y^2 \delta_y^2\right)\left(1-\alpha c^2_{i,j,k} \lambda_z^2 \delta_z^2\right)u_{i,j,k}^{n}=
\notag \\ &
\alpha^2 c^4_{i,j,k} \left(\lambda_x^2 \lambda_y^2 \delta_x^2\delta_y^2+\lambda_x^2 \lambda_z^2 \delta_x^2\delta_z^2
+\lambda_y^2 \lambda_z^2 \delta_y^2\delta_z^2\right)u_{i,j,k}^{n}
-\alpha^3 c^6_{i,j,k} \lambda_x^2 \lambda_y^2 \lambda_z^2 \delta_x^2\delta_y^2\delta_z^2 u_{i,j,k}^{n}\approx O(h^4),
\qquad \qquad (3).
\end{align*}

Furthermore, for simplicity, consider
\begin{align*}
\mathbf{F}_{i,j,k}^{n}&= \left[c_{i,j,k}^2 \beta \left(\lambda_x^2 \delta_x^2 +\lambda_y^2 \delta_y^2 +\lambda_z^2 \delta_z^2 \right)+2\right]u_{i,j,k}^{n}
 +\left[c_{i,j,k}^2\gamma \left(\lambda_x^2 \delta_x^2 +\lambda_y^2 \delta_y^2 +\lambda_z^2 \delta_z^2 \right)-1\right]u_{i,j,k}^{n-1}
+\tau^2f_{i,j,k}^n.
\end{align*}

As a result,
\begin{align*}
\left(1-\alpha c^2_{i,j,k} \lambda_x^2 \delta_x^2\right)\left(1-\alpha c^2_{i,j,k} \lambda_y^2 \delta_y^2\right)\left(1-\alpha c^2_{i,j,k} \lambda_z^2 \delta_z^2\right)u_{i,j,k}^{n+1}=\mathbf{F}_{i,j,k}^{n}.
\end{align*}

### ADI Algorithm

* Approximate $\mathbf{X}_{i,j,k}$ from the following equation for $1\leq i\leq N_x$, $1\leq j\leq N_y$ and $1\leq k\leq N_z$,
\begin{align*}
\left(1-\alpha c^2_{i,j,k} \lambda_x^2 \delta_x^2\right)\mathbf{X}_{i,j,k}=\mathbf{F}_{i,j,k}^{n},
\end{align*}
* Estimate $\mathbf{Y}_{i,j,k}$ from the following system for $1\leq i\leq N_x$, $1\leq j\leq N_y$ and $1\leq k\leq N_z$,
  \begin{align*}
  \left(1-\alpha c^2_{i,j,k} \lambda_y^2 \delta_y^2\right)\mathbf{Y}_{i,j,k}=\mathbf{X}_{i,j,k},
\end{align*}
* Approximate all $u_{i,j,k}^{n+1}$ for the current time step from the following system,
  \begin{align*}
\left(1-\alpha c^2_{i,j,k} \lambda_z^2 \delta_z^2\right)u_{i,j,k}^{n+1}=\mathbf{Y}_{i,j,k}.
\end{align*}

The first time step after the initial time level is needed for proceeding the introduced algorithm and $u(x,y,z,T_1+\tau)$ is approximated by means of Taylor series,
\begin{align*}
u(x,y,z,T_1+\tau)&=
g(x,y,z)+s(x,y,z)\tau+\frac{1}{2}[c^2(x,y,z) \Delta g(x,y,z)+f(x,y,z,T_1)]\tau^2
\notag\\ &
+\frac{1}{6} [c^2(x,y,z) \Delta s(x,y,z)+f_t(x,y,z,T_1)]\tau^3
\notag\\ &
+\frac{1}{24}[c^2(x,y,z) \Delta [c^2(x,y,z) \Delta g(x,y,z)+f(x,y,z,T_1)] +f_{tt}(x,y,z,T_1)]\tau^4
+\mathcal{O}(\tau^5)
\end{align*}

To numerical algorithm based,, first, for current time step, fix $n$ and solve the following system for $i=1,2,\ldots,N_x-1$, $j=1,2,\ldots,N_y-1$ and $k=1,2,\ldots,N_z-1$,
\begin{align*}
\left(1-\alpha c^2_{i,j,k} \lambda_x^2 \delta_x^2\right)\mathbf{X}_{i,j,k}=\mathbf{F}_{i,j,k}^{n},
\qquad \qquad (4)
\end{align*}
In other words,
\begin{align*}\label{eq1C1.11}
-\alpha c_{i,j,k}^2 \lambda_x^2 \left(\mathbf{X}_{i-1,j,k}+\mathbf{X}_{i+1,j,k}\right)+\left(1 +2 \alpha c_{i,j,k}^2 \lambda_x^2\right) \mathbf{X}_{i,j,k} =\mathbf{F}_{i,j,k}^{n}.
\end{align*}
In matrix form:
$$
A_{j,k}=\begin{bmatrix}
1 +2 \alpha c_{1,j,k}^2\lambda_x^2    &   -\alpha c_{1,j,k}^2\lambda_x^2         & 0                      & \dots                   &  \dots         &0
\\
-\alpha c_{2,j,k}^2\lambda_x^2             &1 +2 \alpha c_{2,j,k}^2\lambda_x^2   & -\alpha c_{2,j,k}^2\lambda_x^2           & 0                   &  \vdots         &\vdots
\\
0                       &-\alpha c_{3,j,k}^2\lambda_x^2             &1 +2 \alpha c_{3,j,k}^2\lambda_x^2   & -\alpha c_{3,j,k}^2\lambda_x^2              &0         &\vdots
\\
0                       &0            & \ddots                 & \ddots              &\ddots         &\vdots
\\
\vdots                        &\vdots           & -\alpha c_{N_x-3,j,k}^2\lambda_x^2            & 1 +2 \alpha c_{N_x-3,j,k}^2\lambda_x^2        &-\alpha c_{N_x-3,j,k}^2\lambda_x^2 & 0
\\
\vdots                  &\vdots                      & 0                 & -\alpha c_{N_x-2,j,k}^2\lambda_x^2            & 1 +2 \alpha c_{N_x-2,j,k}^2\lambda_x^2        &-\alpha c^2_{N_x-2,j,k}\lambda_x^2
\\
0                       &0                      & \dots                 & 0               & -\alpha c_{N_x-1,j,k}^2\lambda_x^2    &  1 +2 \alpha c_{N_x-1,j,k}^2\lambda_x^2
\end{bmatrix},
$$
$$
\mathbf{X}_{j,k}=\begin{bmatrix}
\mathbf{X}_{1,j,k} \\
\mathbf{X}_{2,j,k}\\
\vdots\\
\mathbf{X}_{N_x-2,j,k}\\
\mathbf{X}_{N_x-1,j,k}
\end{bmatrix},~
\mathbf{b}_{j,k}^{n}=
\begin{bmatrix}
\alpha c^2_{1,j,k} \lambda_x^2 \mathbf{X}_{0,j,k}\\
0\\
\vdots\\
0\\
\alpha c^2_{N_x-1,j,k}\lambda_x^2 \mathbf{X}_{N_x,j,k},
\end{bmatrix},
\mathbf{F}_{j,k}^{n}=
\begin{bmatrix}
\mathbf{F}_{1,j,k}^{n}\\
\mathbf{F}_{2,j,k}^{n}\\
\vdots\\
\mathbf{F}_{N_x-2,j,k}^{n}\\
\mathbf{F}_{N_x-1,j,k}^{n}
\end{bmatrix}.
$$
where $\mathbf{X}_{0,j,k}$ and $\mathbf{X}_{N_x,j,k}$ for $1\leq j\leq N_x-1$ and $1\leq j\leq N_y-1$ can be stated as follows,
\begin{align*}
\label{eq1C1.12}
\mathbf{X}_{0,j,k}&=
\left(1-\alpha c^2_{i,j,k} \lambda_y^2 \delta_y^2\right)\left(1-\alpha c^2_{i,j,k} \lambda_z^2 \delta_z^2\right)f_{a,j,k}^{n+1},
\\
\label{eq1C1.13}
\mathbf{X}_{N_x,j,k}&=\left(1-\alpha c^2_{i,j,k} \lambda_y^2 \delta_y^2\right)\left(1-\alpha c^2_{i,j,k} \lambda_z^2 \delta_z^2\right)f_{a,j,k}^{n+1}.
\end{align*}
and, $f_{a,j,k}^{n}\approx f_{a}(jh_y,kh_z,n\tau)$ and $f_{b,j,k}^{n}\approx f_{b}(jh_y,kh_z,n\tau)$.

Therefore,
\begin{equation*}
\label{eq1C1.14}\mathbf{X}_{j,k}=A_j^{-1}\left(\mathbf{F}_{j,k}^{n}+\mathbf{b}_j^n\right).
\end{equation*}

Next, we need to solve the following system,
\begin{align*}
\left(1-\alpha c^2_{i,j,k} \lambda_y^2 \delta_y^2\right)\mathbf{Y}_{i,j,k}=\mathbf{X}_{i,j,k}.
\end{align*}
This can be expanded as follows,
\begin{align*}
-\alpha c_{i,j,k}^2 \lambda_y^2 \left(\mathbf{Y}_{i,j-1,k}+\mathbf{Y}_{i,j+1,k}\right)+\left(1 +2 \alpha c_{i,j,k}^2\lambda_y^2\right)\mathbf{Y}_{i,j,k}=\mathbf{X}_{i,j,k}.
\end{align*}
Hence,
\begin{equation*}
\label{eq1C2.13}\mathbf{Y}_{i,k}=A_{i,k}^{-1}\left(\mathbf{X}_{i,k}+\mathbf{b}_{i,k}^n\right),
\end{equation*}
where
\begin{equation*}\label{eq1C2.14}
A_{i,k}=\begin{bmatrix}
1 +2 \alpha c_{i,1,k}^2\lambda_y^2    &   -\alpha c_{i,1,k}^2\lambda_y^2         & 0                      & \dots                   &  \dots         &0
\\
-\alpha c_{i,2,k}^2\lambda_y^2             &1 +2 \alpha c_{i,2,k}^2\lambda_y^2   & -\alpha c_{i,2,k}^2\lambda_y^2           & 0                   &  \vdots         &\vdots
\\
0                       &-\alpha c_{i,3,k}^2\lambda_y^2             &1 +2 \alpha c_{i,3,k}^2\lambda_y^2   & -\alpha c_{i,3,k}^2\lambda_y^2              &0         &\vdots
\\
0                       &0            & \ddots                 & \ddots              &\ddots         &\vdots
\\
\vdots                        &\vdots           & -\alpha c_{i,N_y-3,k}^2\lambda_y^2            & 1 +2 \alpha c_{i,N_y-3,k}^2\lambda_y^2        &-\alpha c_{i,N_y-3,k}^2\lambda_y^2 & 0
\\
\vdots                  &\vdots                      & 0                 & -\alpha c_{i,N_y-2,k}^2\lambda_y^2            & 1 +2 \alpha c_{i,N_y-2,k}^2\lambda_y^2        &-\alpha c^2_{i,N_y-2,k}\lambda_y^2
\\
0                       &0                      & \dots                 & 0               & -\alpha c_{i,N_y-1,k}^2\lambda_y^2    &  1 +2 \alpha c_{i,N_y-1,k}^2\lambda_y^2
\end{bmatrix},
\end{equation*}
and
\begin{equation*}\label{eq1C2.15}
\mathbf{b}_{i,k}^{n}=
\begin{bmatrix}
\alpha c^2_{i,1,k}\lambda_y^2 \mathbf{Y}_{i,0,k}\\
0\\
\vdots\\
0\\
\alpha c^2_{i,N_y-1,k}\lambda_y^2 \mathbf{Y}_{i,N_y,k}
\end{bmatrix},~
\mathbf{X}_{i,k}=\begin{bmatrix}
\mathbf{X}_{i,1,k} \\
\mathbf{X}_{i,2,k}\\
\vdots\\
\mathbf{X}_{i,N_y-1,k}\\
\mathbf{X}_{i,N_y,k}
\end{bmatrix}
\text{ and }
\mathbf{Y}_{i,k}=\begin{bmatrix}
\mathbf{Y}_{i,1,k} \\
\mathbf{Y}_{i,2,k}\\
\vdots\\
\mathbf{Y}_{i,N_y-2,k}\\
\mathbf{Y}_{i,N_y-1,k}
\end{bmatrix}.
\end{equation*}
Moreover, $\mathbf{Y}_{i,0,k}$ and $\mathbf{Y}_{i,N_y,k}$ can be approximated as follows,
\begin{align*}
\label{eq1C1.17}\mathbf{Y}_{i,0,k}&=-\alpha c_{i,0,k}^2 \lambda_z^2 \left(f_{c,i,k-1}^{n+1}+f_{c,i,k+1}^{n+1}\right)+\left(1 +2 \alpha c_{i,0,k}^2\lambda_z^2\right)f_{c,i,k}^{n+1},\\
\label{eq1C1.18}\mathbf{Y}_{i,N_y,k}&=-\alpha c_{i,N_y,k}^2 \lambda_z^2 \left(f_{d,i,k-1}^{n+1}+f_{d,i,k+1}^{n+1}\right)+\left(1 +2 \alpha c_{i,N_y,k}^2\lambda_z^2\right)f_{d,i,k}^{n+1},
\end{align*}
where $f_{c,i,k}^{n}\approx f_{c}(ih_x,kh_z,n\tau)$ and $f_{d,i,k}^{n}\approx f_{d}(ih_x,kh_z,n\tau)$.

Finally, we need to solve the following system for $u_{i,j,k}^{n}$ for each $i=1,2,\ldots,N_x-1$, $j=1,2,\ldots,N_y-1$ and $k=1,2,\ldots,N_z-1$,
\begin{align*}
\label{eq1C1.19}\left(1-\alpha c^2_{i,j,k} \lambda_z^2 \delta_z^2\right)u_{i,j,k}^{n+1}=\mathbf{Y}_{i,j,k}.
\end{align*}
In other words,
\begin{align*}
\label{eq1C1.20}-\alpha c_{i,j,k}^2 \lambda_z^2 \left(u_{i,j,k-1}^{n+1}+u_{i,j,k+1}^{n+1}\right)+\left(1 +2 \alpha c_{i,j,k}^2\lambda_z^2\right)u_{i,j,k}^{n+1}=\mathbf{Y}_{i,j,k}.
\end{align*}
Hence,
\begin{equation*}
\label{eq1C1.21}u_{i,j}^{n+1}=A_{i,j}^{-1}\left(\mathbf{Y}_{i,j,k}+\mathbf{b}_{i,j}^n\right),
\end{equation*}
where
\begin{equation*}\label{eq1C1.22}
A_{i,j}=\begin{bmatrix}
1 +2 \alpha c_{i,j,1}^2\lambda_z^2    &   -\alpha c_{i,j,1}^2\lambda_z^2         & 0                      & \dots                   &  \dots         &0
\\
-\alpha c_{i,j,2}^2\lambda_z^2             &1 +2 \alpha c_{i,j,2}^2\lambda_z^2   & -\alpha c_{i,j,2}^2\lambda_z^2           & 0                   &  \vdots         &\vdots
\\
0                       &-\alpha c_{i,j,3}^2\lambda_z^2             &1 +2 \alpha c_{i,j,3}^2\lambda_z^2   & -\alpha c_{i,j,3}^2\lambda_z^2              &0         &\vdots
\\
0                       &0            & \ddots                 & \ddots              &\ddots         &\vdots
\\
\vdots                        &\vdots           & -\alpha c_{i,j,N_z-3}^2\lambda_z^2            & 1 +2 \alpha c_{i,j,N_z-3}^2\lambda_z^2        &-\alpha c_{i,j,N_z-3}^2\lambda_z^2 & 0
\\
\vdots                  &\vdots                      & 0                 & -\alpha c_{i,j,N_z-2}^2\lambda_z^2            & 1 +2 \alpha c_{i,j,N_z-2}^2\lambda_z^2        &-\alpha c^2_{i,j,N_z-2}\lambda_y^2
\\
0                       &0                      & \dots                 & 0               & -\alpha c_{i,j,N_z-1}^2\lambda_z^2    &  1 +2 \alpha c_{i,j,N_z-1}^2\lambda_y^2
\end{bmatrix},
\end{equation*}
and
\begin{equation*}
\mathbf{b}_{i,j}^{n}=
\begin{bmatrix}
\alpha c^2_{i,j,1}\lambda_z^2 f_{e,i,j}^{n+1}\\
0\\
\vdots\\
0\\
\alpha c^2_{i,j,N_z-1}\lambda_z^2 f_{f,i,j}^{n+1}
\end{bmatrix}
\text{ and }
\mathbf{u}_{i,j}^{n+1}=\begin{bmatrix}
u_{i,j,1}^{n+1} \\
u_{i,j,2}^{n+1}\\
\vdots\\
u_{i,j,N_z-2}^{n+1}\\
u_{i,j,N_z-1}^{n+1}
\end{bmatrix},
\end{equation*}

Note that $f_{e,i,j}^{n}\approx f_{e}(ih_x,jh_y,n\tau)$ and $f_{f,i,j}^{n}\approx f_{e}(ih_x,jh_y,n\tau)$. 

For Stability and convergence analyses, see this [file](https://hatefdastour.github.io/files/FDTD/Stability_and_Convergence_FDTD_2nd_3D.pdf).

## A Fourth-order Scheme

A fourth-order finite difference scheme for solving the problem (1) can be considered as follows,
\begin{align*}
\left(1+\frac{1}{12}\delta_t^2\right)^{-1}\frac{\delta_t^2}{\tau^2} u_{i,j,k}^{n}&=  c^2\left(1+\frac{1}{12}\delta_x^2\right)^{-1}\frac{\delta_x^2}{h_x^2} u_{i,j,k}^{n}+
c^2\left(1+\frac{1}{12}\delta_y^2\right)^{-1}\frac{\delta_y^2}{h_y^2} u_{i,j,k}^{n}\notag \\
&+ c^2\left(1+\frac{1}{12}\delta_z^2\right)^{-1}\frac{\delta_z^2}{h_z^2} u_{i,j,k}^{n}+f_{i,j,k}^{n}.
\end{align*}
Expanding terms,
\begin{align*}
&\left[\left(1+\frac{1}{12}\delta_x^2\right)\left(1+\frac{1}{12}\delta_y^2\right)\left(1+\frac{1}{12}\delta_z^2\right)
-\frac{r_x^2}{12} \left(1+\frac{1}{12}\delta_y^2\right)\left(1+\frac{1}{12}\delta_z^2\right)\delta_x^2
\right. \notag \\ & \left.
-\frac{r_y^2}{12}\left(1+\frac{1}{12}\delta_x^2\right)\left(1+\frac{1}{12}\delta_z^2\right)\delta_y^2
-\frac{r_z^2}{12}\left(1+\frac{1}{12}\delta_x^2\right)\left(1+\frac{1}{12}\delta_y^2\right)\delta_z^2
\right]\delta_t^2u_{i,j,k}^{n}=
\notag \\ &
\left[r_x^2 \left(1+\frac{1}{12}\delta_y^2\right)\left(1+\frac{1}{12}\delta_z^2\right)\delta_x^2
+r_y^2\left(1+\frac{1}{12}\delta_x^2\right)\left(1+\frac{1}{12}\delta_z^2\right)\delta_y^2
+r_z^2\left(1+\frac{1}{12}\delta_x^2\right)\left(1+\frac{1}{12}\delta_y^2\right)\delta_z^2
\right]u_{i,j,k}^{n}
\notag \\ &
+\tau^2\left(1+\frac{1}{12}\delta_x^2\right)\left(1+\frac{1}{12}\delta_y^2\right)\left(1+\frac{1}{12}\delta_z^2\right)\left(1+\frac{1}{12}\delta_t^2\right)f_{i,j,k}^{n},
\end{align*}
where $r_x^2=c^2\frac{\tau^2}{h_x^2}$, $r_y^2=c^2\frac{\tau^2}{h_y^2}$ and $r_z^2=c^2\frac{\tau^2}{h_z^2}$.

Let
\begin{align*}
\rho_x=\frac{1-r_x^2}{12},~\rho_y=\frac{1-r_y^2}{12},~\rho_z=\frac{1-r_z^2}{12}.
\end{align*}

On the other hand, note that,
\begin{align*}
&\left|
\left(1+\frac{1}{12}\delta_x^2\right)\left(1+\frac{1}{12}\delta_y^2\right)\left(1+\frac{1}{12}\delta_z^2\right)u_{i,j,k}^{n}
-\frac{r_x^2}{12} \left(1+\frac{1}{12}\delta_y^2\right)\left(1+\frac{1}{12}\delta_z^2\right)\delta_x^2u_{i,j,k}^{n}
-\frac{r_y^2}{12}\left(1+\frac{1}{12}\delta_x^2\right)\left(1+\frac{1}{12}\delta_z^2\right)\delta_y^2u_{i,j,k}^{n}
\right. \notag \\ & \left.
-\frac{r_z^2}{12}\left(1+\frac{1}{12}\delta_x^2\right)\left(1+\frac{1}{12}\delta_y^2\right)\delta_z^2u_{i,j,k}^{n}
-\left(1+\rho_x\delta_x^2\right)\left(1+\rho_y\delta_y^2\right)\left(1+\rho_z\delta_z^2\right)u_{i,j,k}^{n}
\right|=
\frac{1}{144}\left[r_x^2 r_y^2 \delta_x^2 \delta_y^2
+r_x^2 r_z^2 \delta_x^2 \delta_z^2+r_y^2 r_z^2 \delta_y^2 \delta_z^2\right]u_{i,j,k}^{n}
\notag \\ &
+\frac{1}{1728}\left[r_x^2 r_y^2 +r_x^2 r_z^2 +r_y^2 r_z^2 -r_x^2 r_y^2 r_z^2 \right] \delta_x^2 \delta_y^2 \delta_z^2 u_{i,j,k}^{n}
\approx O(h^4)
\end{align*}

### ADI Algorithm
\begin{align*}
&\left(1+\rho_x\delta_x^2\right)\left(1+\rho_y\delta_y^2\right)\left(1+\rho_z\delta_z^2\right)\delta_t^2 u_{i,j,k}^{n}=
\left[r_x^2 \left(1+\frac{1}{12}\delta_y^2\right)\left(1+\frac{1}{12}\delta_z^2\right)\delta_x^2
\right. \notag \\ & \left.
+r_y^2\left(1+\frac{1}{12}\delta_x^2\right)\left(1+\frac{1}{12}\delta_z^2\right)\delta_y^2
+r_z^2\left(1+\frac{1}{12}\delta_x^2\right)\left(1+\frac{1}{12}\delta_y^2\right)\delta_z^2
\right]u_{i,j,k}^{n}
\notag \\ &
+\tau^2\left(1+\frac{1}{12}\delta_x^2\right)\left(1+\frac{1}{12}\delta_y^2\right)\left(1+\frac{1}{12}\delta_z^2\right)\left(1+\frac{1}{12}\delta_t^2\right)f_{i,j,k}^{n}.
\qquad \qquad (5)
\end{align*}
It follows from (5) that,
\begin{align*}
\left(1+\rho_x\delta_x^2\right)\left(1+\rho_y\delta_y^2\right)\left(1+\rho_z\delta_z^2\right) \delta_t^2 u_{i,j,k}^{n}=\mathbf{F}_{i,j,k}^{n}
\end{align*}
where,
\begin{align*}
\mathbf{F}_{i,j,k}^{n} &=+\left[r_x^2 \left(1+\frac{1}{12}\delta_y^2\right)\left(1+\frac{1}{12}\delta_z^2\right)\delta_x^2
+r_y^2\left(1+\frac{1}{12}\delta_x^2\right)\left(1+\frac{1}{12}\delta_z^2\right)\delta_y^2
\right. \notag \\ & \left.
+r_z^2\left(1+\frac{1}{12}\delta_x^2\right)\left(1+\frac{1}{12}\delta_y^2\right)\delta_z^2
\right]u_{i,j,k}^{n}
\notag \\ &
+\tau^2\left(1+\frac{1}{12}\delta_x^2\right)\left(1+\frac{1}{12}\delta_y^2\right)\left(1+\frac{1}{12}\delta_z^2\right)\left(1+\frac{1}{12}\delta_t^2\right)f_{i,j,k}^{n}.
\end{align*}

The three-dimensional problem can be efficiently solved in three steps using ADI with the following steps,

* Approximate $\mathbf{X}_{i,j,k}$ from the following equation for $1\leq i\leq N_x$, $1\leq j\leq N_y$ and $1\leq j\leq N_z$,
\begin{align*}
\left(1+\rho_x\delta_x^2\right)\mathbf{X}_{i,j,k}=\mathbf{F}_{i,j,k}^{n}.
\qquad \qquad (6)
\end{align*}
* Estimate the solutions of the following system for $1\leq i\leq N_x$, $1\leq j\leq N_y$ and $1\leq j\leq N_z$,
\begin{align*}
\left(1+\rho_y\delta_y^2\right)\mathbf{Y}_{i,j,k}=\mathbf{X}_{i,j,k}.
\end{align*}
* Approximate all $u_{i,j,k}^{n+1}$ for the current time step from the following system,
  \begin{align*}
\left(1+\rho_z\delta_z^2\right)\delta_t^2 u_{i,j,k}^{n}=\mathbf{Y}_{i,j,k}^{n}.
\end{align*}

The equation (6) also can be written as follows,
\begin{align*}
\rho_x(\mathbf{X}_{i+1,j,k}+\mathbf{X}_{i-1,j,k})+(1-2\rho_x)\mathbf{X}_{i,j,k}=\mathbf{F}_{i,j,k}^{n},
\end{align*}

Matrix form:
$$
A_{j,k}=\begin{bmatrix}
(1-2\rho_x)    &   \rho_x         & 0                      & \dots                   &  \dots         &0
\\
\rho_x  &   (1-2\rho_x)   & \rho_x           & 0                   &  \vdots         &\vdots
\\
0               &        \rho_x  &   (1-2\rho_x)   & \rho_x              &0         &\vdots
\\
0                       &0            & \ddots                 & \ddots              &\ddots         &\vdots
\\
\vdots                        &\vdots           & \rho_x  &   (1-2\rho_x)   & \rho_x & 0
\\
\vdots                  &\vdots                      & 0                 & \rho_x  &   (1-2\rho_x)   & \rho_x
\\
0                       &0                      & \dots                 & 0               & \rho_x  &   (1-2\rho_x)
\end{bmatrix},
$$
$$
\mathbf{X}_{j,k}=\begin{bmatrix}
\mathbf{X}_{1,j,k} \\
\mathbf{X}_{2,j,k}\\
\vdots\\
\mathbf{X}_{N_x-2,j,k}\\
\mathbf{X}_{N_x-1,j,k}
\end{bmatrix},~
\mathbf{b}_j^n=
\begin{bmatrix}
-\rho_x \mathbf{X}_{0,j,k}\\
0\\
\vdots\\
0\\
-\rho_x \mathbf{X}_{N_x,j,k},
\end{bmatrix}
\text{ and }
\mathbf{F}_{j}^{n}=
\begin{bmatrix}
\mathbf{F}_{1,j,k}^{n}\\
\mathbf{F}_{2,j,k}^{n}\\
\vdots\\
\mathbf{F}_{N_x-2,j,k}^{n}\\
\mathbf{F}_{N_x-1,j,k}^{n}
\end{bmatrix},
$$
where
\begin{align*}
\mathbf{X}_{0,j,k}&=
\left(1+\rho_y\delta_y^2\right)\left(1+\rho_z\delta_z^2\right)\delta_t^2 f_{a,j,k}^{n},
\\
\mathbf{X}_{N_x,j,k}&=
\left(1+\rho_y\delta_y^2\right)\left(1+\rho_z\delta_z^2\right)\delta_t^2 f_{b,j,k}^{n}.
\end{align*}


Therefore
\begin{equation*}
\mathbf{X}_{j,k}=A_j^{-1}\left(\mathbf{F}_{j,k}^{n}+\mathbf{b}_j^n\right).
\end{equation*}

Next, we need to find $\mathbf{Y}_{i,k}$ for $1\leq i \leq N_x-1$ and $1\leq k\leq N_z-1$. We have
\begin{align*}
\rho_y\left(\mathbf{Y}_{i,j-1,k}+\mathbf{Y}_{i,j+1,k}\right)+(1-2\rho_y)\mathbf{Y}_{i,j,k}=\mathbf{X}_{i,j,k},
\end{align*}

In addition,
$$
A_{i,k}=\begin{bmatrix}
(1-2\rho_y)    &   \rho_y         & 0                      & \dots                   &  \dots         &0
\\
\rho_y  &   (1-2\rho_y)   & \rho_y           & 0                   &  \vdots         &\vdots
\\
0                       & \rho_y  &   (1-2\rho_y)   & \rho_y              &0         &\vdots
\\
0                       &0            & \ddots                 & \ddots              &\ddots         &\vdots
\\
\vdots                        &\vdots           & \rho_y  &   (1-2\rho_y)   & \rho_y & 0
\\
\vdots                  &\vdots                      & 0                 & \rho_y  &   (1-2\rho_y)   & \rho_y
\\
0                       &0                      & \dots                 & 0               & \rho_y  &   (1-2\rho_y)
\end{bmatrix},
$$
and
\begin{equation*}
\mathbf{b}_{i,k}^{n}=
\begin{bmatrix}
-\rho_y \mathbf{Y}_{i,0,k}\\
0\\
\vdots\\
0\\
-\rho_y \mathbf{Y}_{i,N_y,k}
\end{bmatrix},~
\mathbf{X}_{i,k}=\begin{bmatrix}
\mathbf{X}_{i,1,k} \\
\mathbf{X}_{i,2,k}\\
\vdots\\
\mathbf{X}_{i,N_y-2,k}\\
\mathbf{X}_{i,N_y-1,k}
\end{bmatrix}
\text{ and }
\mathbf{Y}_{i,k}=\begin{bmatrix}
\mathbf{Y}_{i,1,k} \\
\mathbf{Y}_{i,2,k}\\
\vdots\\
\mathbf{Y}_{i,N_y-2,k}\\
\mathbf{Y}_{i,N_y-1,k}
\end{bmatrix},
\end{equation*}
where
\begin{align*}
\mathbf{Y}_{i,0,k}&=\left(1+\rho_z\delta_z^2\right)\delta_t^2f_{c,i,k}^{n},\\
\mathbf{Y}_{i,N_y,k}&=\left(1+\rho_z\delta_z^2\right)\delta_t^2 f_{d,i,k}^{n}.
\end{align*}

Hence,
\begin{equation*}
\label{eq1C6.15-1}\mathbf{Y}_{i,k}=A_{i,k}^{-1}\left(\mathbf{X}_{i,k}+\mathbf{b}_{i,k}^n\right),
\end{equation*}

Finally, solving the following linear system is needed to find $\mathbf{Z}_{i,j,k}$

\begin{align*}
\begin{cases}
\rho_z( \mathbf{Z}_{i,j,k+1}+\mathbf{Z}_{i,j,k-1})+(1-2\rho_z)\mathbf{Z}_{i,j,k}=\mathbf{Y}_{i,j,k},\\
u_{i,j,k+1}^{n+1}=\mathbf{Z}_{i,j,k}+2u_{i,j,k}^{n}-u_{i,j,k}^{n-1}
\end{cases}
\end{align*}

As a result,
\begin{equation*}
\mathbf{u}_{i,j}^{n+1}=A_{i,j}^{-1}\left(\mathbf{Y}_{i,j}+\mathbf{b}_{i,j}^n\right),
\end{equation*}
where
\begin{equation*}
A_{i,j}=\begin{bmatrix}
(1-2\rho_z)    &   \rho_z         & 0                      & \dots                   &  \dots         &0
\\
\rho_z  &   (1-2\rho_z)   & \rho_z           & 0                   &  \vdots         &\vdots
\\
0                       & \rho_z  &   (1-2\rho_z)   & \rho_z              &0         &\vdots
\\
0                       &0            & \ddots                 & \ddots              &\ddots         &\vdots
\\
\vdots                        &\vdots           & \rho_z  &   (1-2\rho_z)   & \rho_z & 0
\\
\vdots                  &\vdots                      & 0                 & \rho_z  &   (1-2\rho_z)   & \rho_z
\\
0                       &0                      & \dots                 & 0               & \rho_z  &   (1-2\rho_z)
\end{bmatrix},
\end{equation*}
and
\begin{equation*}
\mathbf{b}_{i,j}^{n}=
\begin{bmatrix}
-\rho_z \delta_t^2 f_{e,i,j}^{n}\\
0\\
\vdots\\
0\\
-\rho_z \delta_t^2 f_{f,i,j}^{n}
\end{bmatrix}
\text{ and }
\mathbf{u}_{i,j}^{n+1}=\begin{bmatrix}
u_{i,j,1}^{n+1} \\
u_{i,j,2}^{n+1}\\
\vdots\\
u_{i,j,N_z-2}^{n+1}\\
u_{i,j,N_z-1}^{n+1}
\end{bmatrix}.
\end{equation*} 

For stability, convergence and dispersion analyses, please see [[Das, 2012](#https://www.sciencedirect.com/science/article/pii/S0377042713004548), [Liao, 2014](#https://www.sciencedirect.com/science/article/pii/S0377042713004299)].

## Example 1
Consider the following problem,
\begin{align*}
\begin{cases}
\dfrac{\partial^2 u}{\partial t^2} = \frac{1}{3}e^{x+y+z} \Delta u +e^{-t-x-y-z}-e^{-t},&(x,y,z)\in I,~t\in(0,1],\\
u(x,y,z,0)=g(x,y,z),&(x,y,z)\in J,\\
\dfrac{\partial }{\partial t}u(x,y,z,1) = s(x,y,z),&(x,y,z)\in J,\\
u(0,y,z,t) = e^{-t-y-z},&(y,z)\in [0,1]\times[0,1],~t \in[0,1],\\
u(1,y,z,t) = e^{-t-y-z-1},&(y,z)\in [0,1]\times[0,1],~t \in[0,1],\\
u(x,0,z,t) = e^{-t-x-z},&(x,z)\in [0,1]\times[0,1],~t \in[0,1],\\
u(x,1,z,t) = e^{-t-x-z-1},&(x,z)\in [0,1]\times[0,1],~t \in[0,1],\\
u(x,y,0,t) = e^{-t-x-y},&(x,y)\in [0,1]\times[0,1],~t \in[0,1],\\
u(x,y,1,t) = e^{-t-x-y-1},&(x,y)\in [0,1]\times[0,1],~t \in[0,1].
\end{cases}
\end{align*} 

The exact solution corresponding to the problem can be found as follows,
$$
u(x,t)=e^{-t-x-y-z}
$$ 

### The Explicit Scheme

In [52]:
def Example01_ExSolver3D(N,Nt):
    # N = the number of mesh points for space
    # Nt = the number of mesh points for time
    Nx=N
    Ny=N
    Nz=N
    # (x,y,z) \in [a_1,a_2]*[b_1,b_2]*[c_1,c_2]
    a1 = 0.0
    a2 = 1.0
    b1 = 0.0
    b2 = 1.0
    c1 = 0.0
    c2 = 1.0
    # t \in [T_1,T_2]
    T1=0
    T2=1
    # Delta x, Delta y, Delta z and Delta t
    hx =(a2-a1)/Nx
    hy =(b2-b1)/Ny
    hz =(c2-c1)/Nz
    ht =(T2-T1)/Nt
    # \lambda_x, \lambda_y and \lambda_z
    lx2=(ht/hx)**2
    ly2=(ht/hy)**2
    lz2=(ht/hz)**2
    # Initial and Boundary Conditions
    C2 =lambda x,y,z: 1/3*np.exp(x+y+z)
    g=lambda x,y,z: np.exp(-x-y-z)
    fa=lambda y,z,t: np.exp(-y-z-t)
    fb=lambda y,z,t: np.exp(-y-z-t-1)
    fc=lambda x,z,t: np.exp(-x-z-t)
    fd=lambda x,z,t: np.exp(-x-z-t-1)
    fe=lambda x,y,t: np.exp(-x-y-t)
    ff=lambda x,y,t: np.exp(-x-y-t-1)
    f=lambda x,y,z,t: np.exp(- t - x - y - z) - np.exp(-t)
    # the exact solution
    Ue=lambda x,y,z,t: np.exp(-x-y-z-t)
    # Null Matrices
    CC=np.zeros((Nx+1,Ny+1,Nz+1), dtype=float)
    Uexact=np.zeros((Nx+1,Ny+1,Nz+1), dtype=float)
    u0=np.zeros((Nx+1,Ny+1,Nz+1), dtype=float)
    u1=np.zeros((Nx+1,Ny+1,Nz+1), dtype=float)
    # x, y, z and t
    xx=np.linspace(a1, a2, Nx+1)
    yy=np.linspace(b1, b2, Ny+1)
    zz=np.linspace(c1, c2, Nz+1)
    tt=np.linspace(T1, T2, Nt+1)

    for k in range(Nz+1):
        for j in range(Ny+1):
            CC[:,j,k] = C2(xx,yy[j],zz[k])
            Uexact[:,j,k] = Ue(xx,yy[j],zz[k],T2)
            u0[:,j,k] = g(xx,yy[j],zz[k])

    # indeces
    mid=list(range(1,Nx))
    midp=[i+1 for i in mid]
    midm=[i-1 for i in mid]
    # Computing the solution at the first time step
    GG=lambda x,y,z: np.exp(-x-y-z)-np.exp(-x-y-z)*ht\
    +(0.5)*(1+ np.exp(- T1 - x - y - z) - np.exp(-T1))*ht**2\
    +(1/6)*(-1 - np.exp(- T1 - x - y - z) + np.exp(-T1))*ht**3\
    +(1/24)*(1+ np.exp(- T1 - x - y - z) - np.exp(-T1))*ht**4
    for k in mid:
        for j in mid:
            u1[mid,j,k]=GG(xx[mid],yy[j],zz[k])

    for j in range(Ny+1):
        u1[:,j,0]=fe(xx,yy[j],tt[1])
        u1[:,j,Nz]=ff(xx,yy[j],tt[1])

    for k in range(Nz+1):
        u1[0,:,k]=fa(yy,zz[k],tt[1])
        u1[Nx,:,k]=fb(yy,zz[k],tt[1])
        u1[:,0,k]=fc(xx,zz[k],tt[1])
        u1[:,Ny,k]=fd(xx,zz[k],tt[1])
    # for loop (n>2)
    for n in range(1,Nt):
        u=np.zeros((Nx+1,Ny+1,Nz+1), dtype=float)
        for k in mid:
            for j in mid:
                u[mid,j,k]=lx2*CC[mid,j,k]*u1[midp,j,k]\
                    +lx2*CC[mid,j,k]*u1[midm,j,k]\
                    +ly2*CC[mid,j,k]*u1[mid,j+1,k]\
                    +ly2*CC[mid,j,k]*u1[mid,j-1,k]\
                    +lz2*CC[mid,j,k]*u1[mid,j,k+1]\
                    +lz2*CC[mid,j,k]*u1[mid,j,k-1]\
                    +2*u1[mid,j,k]-2*lx2*CC[mid,j,k]*u1[mid,j,k]\
                    -2*ly2*CC[mid,j,k]*u1[mid,j,k]\
                    -2*lz2*CC[mid,j,k]*u1[mid,j,k]-u0[mid,j,k]\
                    +(ht**2)*f(xx[mid],yy[j],zz[k],tt[n])
        for k in range(Nz+1):
            for j in range(Ny+1):
                u[:,j,0]=fe(xx,yy[j],tt[n+1])
                u[:,j,Nz]=ff(xx,yy[j],tt[n+1])
            u[0,:,k]=fa(yy,zz[k],tt[n+1])
            u[Nx,:,k]=fb(yy,zz[k],tt[n+1])
            u[:,0,k]=fc(xx,zz[k],tt[n+1])
            u[:,Ny,k]=fd(xx,zz[k],tt[n+1])
        u0=u1
        u1=u
    # Norm
    Norm=np.max(np.abs(u-Uexact))
    return Norm,u,Uexact

In [53]:
it=3
Norm=np.zeros(it, dtype=float)
N0=10;
Nt0=200;
N=np.asarray([N0*2**n for n in range(0,it)])
Nt=np.asarray([Nt0 for n in range(0,it)])
Ratio=np.zeros(it, dtype=float)
LOG=np.zeros(it, dtype=float)
# iteration
for n in range(it):
    Norm[n],_,_ =Example01_ExSolver3D(N[n],Nt[n])
    if (n>0):
        Ratio[n]=Norm[n-1]/Norm[n]
        LOG[n]=math.log(Ratio[n],2)
        
data = pd.DataFrame({'Nx': N, 'Ny': N, 'Nz': N, 'Nt': Nt, 'Norm': Norm, 'Ratio': Ratio, 'Log': LOG})
data

,Nx,Ny,Nz,Nt,Norm,Ratio,Log
0,10,10,10,200,0.000030,0.000000,0.000000
1,20,20,20,200,0.000007,4.523345,2.177390
2,40,40,40,200,0.000002,4.080920,2.028894


In [56]:
it=3
Norm=np.zeros(it, dtype=float)
N0=5;
Nt0=20;
N=np.asarray([N0*2**n for n in range(0,it)])
Nt=np.asarray([Nt0*2**n for n in range(0,it)])
Ratio=np.zeros(it, dtype=float)
LOG=np.zeros(it, dtype=float)
# iteration
for n in range(it):
    Norm[n],_,_ =Example01_ExSolver3D(N[n],Nt[n])
    if (n>0):
        Ratio[n]=Norm[n-1]/Norm[n]
        LOG[n]=math.log(Ratio[n],2)
        
data = pd.DataFrame({'Nx': N, 'Ny': N, 'Nz': N, 'Nt': Nt, 'Norm': Norm, 'Ratio': Ratio, 'Log': LOG})
data

,Nx,Ny,Nz,Nt,Norm,Ratio,Log
0,5,5,5,20,0.000134,0.000000,0.000000
1,10,10,10,40,0.000030,4.482758,2.164387
2,20,20,20,80,0.000007,4.507186,2.172227


What stands out from the tables is that the order of the method is 2 (in space and time). Moreover, consider the following experiment to test the stability condition.

In [57]:
it=3
Norm=np.zeros(it, dtype=float)
N0=5;
Nt0=3;
N=np.asarray([N0*2**n for n in range(0,it)])
Nt=np.asarray([Nt0*2**n for n in range(0,it)])
Ratio=np.zeros(it, dtype=float)
LOG=np.zeros(it, dtype=float)
# iteration
for n in range(it):
    Norm[n],_,_ =Example01_ExSolver3D(N[n],Nt[n])
    if (n>0):
        Ratio[n]=Norm[n-1]/Norm[n]
        LOG[n]=math.log(Ratio[n],2)
        
data = pd.DataFrame({'Nx': N, 'Ny': N, 'Nz': N, 'Nt': Nt, 'Norm': Norm, 'Ratio': Ratio, 'Log': LOG})
data

,Nx,Ny,Nz,Nt,Norm,Ratio,Log
0,5,5,5,3,3.364656e-03,0.000000e+00,0.000000
1,10,10,10,6,1.352591e+02,2.487564e-05,-15.294907
2,20,20,20,12,3.100136e+13,4.363004e-12,-37.737815


### The Implicit Scheme

In [2]:
def Example01_ImSolver3D(N,Nt):
    # alpha, beta and gamma
    alpha=1/4
    beta=1/2
    gamma=1/4
    # N = the number of mesh points for space
    # Nt = the number of mesh points for time
    Nx=N
    Ny=N
    Nz=N
    # (x,y,z) \in [a_1,a_2]*[b_1,b_2]*[c_1,c_2]
    a1 = 0.0
    a2 = 1.0
    b1 = 0.0
    b2 = 1.0
    c1 = 0.0
    c2 = 1.0
    # t \in [T_1,T_2]
    T1=0
    T2=1
    # Delta x, Delta y, Delta z and Delta t
    hx =(a2-a1)/Nx
    hy =(b2-b1)/Ny
    hz =(c2-c1)/Nz
    ht =(T2-T1)/Nt
    # \lambda_x, \lambda_y and \lambda_z
    lx2=(ht/hx)**2
    ly2=(ht/hy)**2
    lz2=(ht/hz)**2
    # Initial and Boundary Conditions
    C2 =lambda x,y,z: 1/3*np.exp(x+y+z)
    g=lambda x,y,z: np.exp(-x-y-z)
    fa=lambda y,z,t: np.exp(-y-z-t)
    fb=lambda y,z,t: np.exp(-y-z-t-1)
    fc=lambda x,z,t: np.exp(-x-z-t)
    fd=lambda x,z,t: np.exp(-x-z-t-1)
    fe=lambda x,y,t: np.exp(-x-y-t)
    ff=lambda x,y,t: np.exp(-x-y-t-1)
    f=lambda x,y,z,t: np.exp(- t - x - y - z) - np.exp(-t)
    # the exact solution
    Ue=lambda x,y,z,t: np.exp(-x-y-z-t)
    # Null Matrices
    CC=np.zeros((Nx+1,Ny+1,Nz+1), dtype=float)
    Uexact=np.zeros((Nx+1,Ny+1,Nz+1), dtype=float)
    u0=np.zeros((Nx+1,Ny+1,Nz+1), dtype=float)
    u1=np.zeros((Nx+1,Ny+1,Nz+1), dtype=float)
    # x, y, z and t
    xx=np.linspace(a1, a2, Nx+1)
    yy=np.linspace(b1, b2, Ny+1)
    zz=np.linspace(c1, c2, Nz+1)
    tt=np.linspace(T1, T2, Nt+1)

    for k in range(Nz+1):
        for j in range(Ny+1):
            CC[:,j,k] = C2(xx,yy[j],zz[k])
            Uexact[:,j,k] = Ue(xx,yy[j],zz[k],T2)
            u0[:,j,k] = g(xx,yy[j],zz[k])
    # indeces
    mid=list(range(1,Nx))
    midp=[i+1 for i in mid]
    midm=[i-1 for i in mid]
    sup=list(range(1,Nx-1))
    sub=list(range(2,Nx))
    # Computing the solution at the first time step
    GG=lambda x,y,z: np.exp(-x-y-z)-np.exp(-x-y-z)*ht\
    +(0.5)*(1+ np.exp(- T1 - x - y - z) - np.exp(-T1))*ht**2\
    +(1/6)*(-1 - np.exp(- T1 - x - y - z) + np.exp(-T1))*ht**3\
    +(1/24)*(1+ np.exp(- T1 - x - y - z) - np.exp(-T1))*ht**4
    for k in mid:
        for j in mid:
            u1[mid,j,k]=GG(xx[mid],yy[j],zz[k])

    for j in range(Ny+1):
        u1[:,j,0]=fe(xx,yy[j],tt[1])
        u1[:,j,Nz]=ff(xx,yy[j],tt[1])

    for k in range(Nz+1):
        u1[0,:,k]=fa(yy,zz[k],tt[1])
        u1[Nx,:,k]=fb(yy,zz[k],tt[1])
        u1[:,0,k]=fc(xx,zz[k],tt[1])
        u1[:,Ny,k]=fd(xx,zz[k],tt[1])
    # for loop (n>2)
    for n in range(1,Nt):
        u=np.zeros((Nx+1,Ny+1,Nz+1), dtype=float)
        X=np.zeros((Nx+1,Ny+1,Nz+1), dtype=float)
        Y=np.zeros((Nx+1,Ny+1,Nz+1), dtype=float)
        # Part 1: Finding the values of Y for current time step
        # f_{a,j,k]}^{n+1} and f_{b,j,k]}^{n+1}
        FA=np.zeros((Ny+1,Nz+1), dtype=float)
        FB=np.zeros((Ny+1,Nz+1), dtype=float)
        for k in range(Nz+1):
            FA[:,k]=fa(yy,zz[k],tt[n+1])
            FB[:,k]=fb(yy,zz[k],tt[n+1])
        for k in mid:
            for j in mid:
                # Matrix A_{j,k]}
                Ajk=diags([-alpha*CC[sub,j,k]*lx2,1+2*alpha*CC[mid,j,k]*lx2,-alpha*CC[sup,j,k]*lx2], [-1,0,1])
                # F_{j,k}
                Fjk=lx2*beta*CC[mid,j,k]*(u1[midp,j,k]+u1[midm,j,k])\
                    +ly2*beta*CC[mid,j,k]*(u1[mid,j+1,k]+u1[mid,j-1,k])\
                    +lz2*beta*CC[mid,j,k]*(u1[mid,j,k+1]+u1[mid,j,k-1])\
                    +(2-2*beta*CC[mid,j,k]*(lx2+ly2+lz2))*u1[mid,j,k]\
                    +lx2*gamma*CC[mid,j,k]*(u0[midp,j,k]+u0[midm,j,k])\
                    +ly2*gamma*CC[mid,j,k]*(u0[mid,j+1,k]+u0[mid,j-1,k])\
                    +lz2*gamma*CC[mid,j,k]*(u0[mid,j,k+1]+u0[mid,j,k-1])\
                    -(1+2*gamma*CC[mid,j,k]*(lx2+ly2+lz2))*u0[mid,j,k]\
                    +((ht**2)*f(xx[mid],yy[j],zz[k],tt[n]))
                #
                X_0jk=(alpha*alpha)*CC[0,j,k]*ly2*lz2*(CC[0,j+1,k]*(FA[j+1,k+1]+FA[j+1,k-1])\
                    +CC[0,j-1,k]*(FA[j-1,k+1]+FA[j-1,k-1]))\
                    -alpha*CC[0,j,k]*lz2*(1+2*alpha*CC[0,j,k]*ly2)*(FA[j,k+1]+FA[j,k-1])\
                    -alpha*CC[0,j,k]*ly2*((1+2*alpha*CC[0,j+1,k]*lz2)*FA[j+1,k]+(1+2*alpha*CC[0,j-1,k]*lz2)*FA[j-1,k])\
                    +(1+2*alpha*CC[0,j,k]*ly2)*(1+2*alpha*CC[0,j,k]*lz2)*FA[j,k]
                #
                X_Nxjk=(alpha*alpha)*CC[Nx,j,k]*ly2*lz2*(CC[Nx,j+1,k]*(FB[j+1,k+1]+FB[j+1,k-1])\
                    +CC[Nx,j-1,k]*(FB[j-1,k+1]+FB[j-1,k-1]))\
                    -alpha*CC[Nx,j,k]*lz2*(1+2*alpha*CC[Nx,j,k]*ly2)*(FB[j,k+1]+FB[j,k-1])\
                    -alpha*CC[Nx,j,k]*ly2*((1+2*alpha*CC[Nx,j+1,k]*lz2)*FB[j+1,k]+(1+2*alpha*CC[Nx,j-1,k]*lz2)*FB[j-1,k])\
                    +(1+2*alpha*CC[Nx,j,k]*ly2)*(1+2*alpha*CC[Nx,j,k]*lz2)*FB[j,k]
                # b_{j,k}
                bjk=np.zeros(len(mid), dtype=float)
                bjk[0]=alpha*CC[1,j,k]*lx2*X_0jk
                bjk[Nx-2]=alpha*CC[Nx-1,j,k]*lx2*X_Nxjk
                # X
                X[mid,j,k]=spsolve(Ajk,Fjk+bjk, use_umfpack=True)
                del Ajk, bjk, Fjk, X_0jk, X_Nxjk
        # Part 2: Finding the values of Y for current time step
        FC=np.zeros((Nx+1,Nz+1), dtype=float)
        FD=np.zeros((Nx+1,Nz+1), dtype=float)
        for k in range(Nz+1):
            # f_{c,j,k}^{n+1} and f_{d,j,k}^{n+1}
            FC[:,k]= fc(xx,zz[k],tt[n+1])
            FD[:,k]= fd(xx,zz[k],tt[n+1])
        for k in mid:
            for i in mid:
                # Matrix A_{i,k}
                Aik=diags([-alpha*CC[i,sub,k]*ly2,1+2*alpha*CC[i,mid,k]*ly2,-alpha*CC[i,sup,k]*ly2], [-1,0,1])
                Y_i0k=-alpha*CC[i,0,k]*lz2*(FC[i,k-1]+FC[i,k+1])+(1+2*alpha*CC[i,0,k]*lz2)*FC[i,k]
                Y_iNyj=-alpha*CC[i,Ny,k]*lz2*(FD[i,k-1]+FD[i,k+1])+(1+2*alpha*CC[i,Ny,k]*lz2)*FD[i,k]
                # b_{i,k]
                bik=np.zeros(len(mid), dtype=float)
                bik[0]=alpha*CC[i,1,k]*ly2*Y_i0k
                bik[Ny-2]=alpha*CC[i,Ny-1,k]*ly2*Y_iNyj
                Y[i,mid,k]=spsolve(Aik,X[i,mid,k]+bik, use_umfpack=True)
                del Aik, bik, Y_i0k, Y_iNyj
        # Part 3: Finding the values of u^{n+1} for current time step
        FE=np.zeros((Nx+1,Ny+1), dtype=float)
        FF=np.zeros((Nx+1,Ny+1), dtype=float)
        for j in range(Ny+1):
            # f_{e,i,j}^{n+1} and f_{f,i,j}^{n+1}
            FE[:,j]= fc(xx,yy[j],tt[n+1])
            FF[:,j]= fd(xx,yy[j],tt[n+1])
        for j in mid:
            for i in mid:
                # Matrix A_{i,j}
                Aik=diags([-alpha*CC[i,j,sub]*lz2,1+2*alpha*CC[i,j,mid]*lz2,-alpha*CC[i,j,sup]*lz2], [-1,0,1])
                bij=np.zeros(len(mid), dtype=float)
                bij[0]=alpha*CC[i,j,1]*ly2*FE[i,j]
                bij[Nz-2]=alpha*CC[i,j,Nz-1]*ly2*FF[i,j]
                u[i,j,mid]=spsolve(Aik,Y[i,j,mid]+bij, use_umfpack=True)
                del Aik, bij
        u[0,:,:]=FA
        u[Nx,:,:]=FB
        u[:,0,:]=FC
        u[:,Ny,:]=FD
        u[:,:,0]=FE
        u[:,:,Nz]=FF
        u0=u1
        u1=u
    # Norm
    Norm=np.max(np.abs(u-Uexact))
    return Norm,u,Uexact

In [103]:
it=3
Norm=np.zeros(it, dtype=float)
N0=5
Nt0=100
N=np.asarray([N0*2**n for n in range(0,it)])
Nt=np.asarray([Nt0 for n in range(0,it)])
Ratio=np.zeros(it, dtype=float)
LOG=np.zeros(it, dtype=float)
# iteration
for n in range(it):
    Norm[n],_,_ =Example01_ImSolver3D(N[n],Nt[n])
    if (n>0):
        Ratio[n]=Norm[n-1]/Norm[n]
        LOG[n]=math.log(Ratio[n],2)
        
data = pd.DataFrame({'Nx': N, 'Ny': N, 'Nz': N, 'Nt': Nt, 'Norm': Norm, 'Ratio': Ratio, 'Log': LOG})
data

,Nx,Ny,Nz,Nt,Norm,Ratio,Log
0,5,5,5,100,0.000139,0.000000,0.000000
1,10,10,10,100,0.000031,4.451670,2.154347
2,20,20,20,100,0.000007,4.202264,2.071167


## Example 2
Consider the following problem,
\begin{align*}
\begin{cases}
\dfrac{\partial^2 u}{\partial t^2} = \frac{1}{4} \Delta u +\frac{1}{4}e^{t+x+y+z},&(x,y,z)\in I,~t\in(0,1],\\
u(x,y,z,0)=g(x,y,z),&(x,y,z)\in J,\\
\dfrac{\partial }{\partial t}u(x,y,z,0) = s(x,y,z),&(x,y,z)\in J,\\
u(0,y,z,t) = e^{t+y+z},&(y,z)\in [0,1]\times[0,1],~t \in[0,1],\\
u(1,y,z,t) = e^{t+y+z+1},&(y,z)\in [0,1]\times[0,1],~t \in[0,1],\\
u(x,0,z,t) = e^{t+x+z},&(x,z)\in [0,1]\times[0,1],~t \in[0,1],\\
u(x,1,z,t) = e^{t+x+z+1},&(x,z)\in [0,1]\times[0,1],~t \in[0,1],\\
u(x,y,0,t) = e^{t+x+y},&(x,y)\in [0,1]\times[0,1],~t \in[0,1],\\
u(x,y,1,t) = e^{t+x+y+1},&(x,y)\in [0,1]\times[0,1],~t \in[0,1].
\end{cases}
\end{align*} 

The exact solution corresponding to the problem can be found as follows,
$$
u(x,t)=e^{t+x+y+z}
$$ 

### The Fourth-order Scheme

We can consider one of the following algorithms for this scheme.

In [90]:
def Example02_4thSolver3D(N,Nt):
    # N = the number of mesh points for space
    # Nt = the number of mesh points for time
    Nx=N
    Ny=N
    Nz=N
    # (x,y,z) \in [a_1,a_2]*[b_1,b_2]*[c_1,c_2]
    a1 = 0.0
    a2 = 1.0
    b1 = 0.0
    b2 = 1.0
    c1 = 0.0
    c2 = 1.0
    # t \in [T_1,T_2]
    T1=0
    T2=1
    # Delta x, Delta y, Delta z and Delta t
    hx =(a2-a1)/Nx
    hy =(b2-b1)/Ny
    hz =(c2-c1)/Nz
    ht =(T2-T1)/Nt
    # \lambda_x, \lambda_y and \lambda_z
    lx2=(ht/hx)**2
    ly2=(ht/hy)**2
    lz2=(ht/hz)**2
    # Initial and Boundary Conditions
    C2 =1/4
    g=lambda x,y,z: np.exp(x+y+z)
    fa=lambda y,z,t: np.exp(y+z+t)
    fb=lambda y,z,t: np.exp(y+z+t+1)
    fc=lambda x,z,t: np.exp(x+z+t)
    fd=lambda x,z,t: np.exp(x+z+t+1)
    fe=lambda x,y,t: np.exp(x+y+t)
    ff=lambda x,y,t: np.exp(x+y+t+1)
    f=lambda x,y,z,t: np.exp(x+y+z+t)/4
    # the exact solution
    Ue=lambda x,y,z,t: np.exp(x+y+z+t)
    # Additional Constants 
    rx2=C2*(ht/hx)**2
    ry2=C2*(ht/hy)**2
    rz2=C2*(ht/hz)**2
    px=(1-rx2)/12
    py=(1-ry2)/12
    pz=(1-rz2)/12
    # indeces
    mid=list(range(1,Nx))
    midp=[i+1 for i in mid]
    midm=[i-1 for i in mid]
    sup=list(range(1,Nx-1))
    sub=list(range(2,Nx))
    # Null Matrices
    Uexact=np.zeros((Nx+1,Ny+1,Nz+1), dtype=float)
    u0=np.zeros((Nx+1,Ny+1,Nz+1), dtype=float)
    u1=np.zeros((Nx+1,Ny+1,Nz+1), dtype=float)
    F=np.zeros((Nx+1,Ny+1,Nz+1,Nt+1), dtype=float)
    FA=np.zeros((Ny+1,Nz+1,Nt+1), dtype=float)
    FB=np.zeros((Ny+1,Nz+1,Nt+1), dtype=float)
    FC=np.zeros((Nx+1,Nz+1,Nt+1), dtype=float)
    FD=np.zeros((Nx+1,Nz+1,Nt+1), dtype=float)
    FE=np.zeros((Nx+1,Nz+1,Nt+1), dtype=float)
    FF=np.zeros((Nx+1,Nz+1,Nt+1), dtype=float)
    # x, y, z and t
    xx=np.linspace(a1, a2, Nx+1)
    yy=np.linspace(b1, b2, Ny+1)
    zz=np.linspace(c1, c2, Nz+1)
    tt=np.linspace(T1, T2, Nt+1)
    #
    for n in range(Nt+1):
        for k in range(Nz+1):
            for j in range(Ny+1):
                F[:,j,k,n] = f(xx,yy[j],zz[k],tt[n])
                Uexact[:,j,k] = Ue(xx,yy[j],zz[k],T2)
                u0[:,j,k] = g(xx,yy[j],zz[k])
            # f{a,j,k}^{n+1} and f{b,j,k}^{n+1}
            FA[:,k,n]=fa(yy,zz[k],tt[n])
            FB[:,k,n]=fb(yy,zz[k],tt[n])
            # f{c,i,k}^{n+1} and f{d,i,k}^{n+1}
            FC[:,k,n]= fc(xx,zz[k],tt[n])
            FD[:,k,n]= fd(xx,zz[k],tt[n])
        for j in range(Ny+1):
            # f{e,i,j}^{n+1} and f{f,i,j}^{n+1}
            FE[:,j,n]= fc(xx,yy[j],tt[n])
            FF[:,j,n]= fd(xx,yy[j],tt[n])
    # Computing the solution at the first time step
    GG=lambda x,y,z: np.exp(x+y+z+T1)+np.exp(x+y+z+T1)*ht+(0.5)*np.exp(x+y+z+T1)*ht**2\
        +(1/6)*np.exp(x+y+z+T1)*ht**3+(1/24)*np.exp(x+y+z+T1)*ht**4
    for k in mid:
        for j in mid:
            u1[mid,j,k]=GG(xx[mid],yy[j],zz[k])
    del g, T1, T2
    for j in range(Ny+1):
        u1[:,j,0]=fe(xx,yy[j],tt[1])
        u1[:,j,Nz]=ff(xx,yy[j],tt[1])
    for k in range(Nz+1):
        u1[0,:,k]=fa(yy,zz[k],tt[1])
        u1[Nx,:,k]=fb(yy,zz[k],tt[1])
        u1[:,0,k]=fc(xx,zz[k],tt[1])
        u1[:,Ny,k]=fd(xx,zz[k],tt[1])
    # Marices Ajk, Aik
    Ajk = diags([px*np.ones(Nx-2),(1-2*px)*np.ones(Nx-1),px*np.ones(Nx-2)], [-1,0,1])
    Aik = diags([py*np.ones(Ny-2),(1-2*py)*np.ones(Ny-1),py*np.ones(Ny-2)], [-1,0,1])
    Aij = diags([pz*np.ones(Nz-2),(1-2*pz)*np.ones(Nz-1),pz*np.ones(Nz-2)], [-1,0,1])
    # for loop (n>2)
    for n in range(1,Nt):
        u=np.zeros((Nx+1,Ny+1,Nz+1), dtype=float)
        X=np.zeros((Nx+1,Ny+1,Nz+1), dtype=float)
        Y=np.zeros((Nx+1,Ny+1,Nz+1), dtype=float)
        Z=np.zeros((Nx+1,Ny+1,Nz+1), dtype=float)
        # Part 1: Finding the values of Y for current time step
        for k in mid:
            for j in mid:
                A2=(1/144)*rx2*(u1[midp,j+1,k+1]-2*u1[mid,j+1,k+1]+u1[midm,j+1,k+1])+\
                    (10/144)*rx2*(u1[midp,j+1,k]-2*u1[mid,j+1,k]+u1[midm,j+1,k])+\
                    (1/144)*rx2*(u1[midp,j+1,k-1]-2*u1[mid,j+1,k-1]+u1[midm,j+1,k-1])+\
                    (10/144)*rx2*(u1[midp,j,k+1]-2*u1[mid,j,k+1]+u1[midm,j,k+1])+\
                    (100/144)*rx2*(u1[midp,j,k]-2*u1[mid,j,k]+u1[midm,j,k])+\
                    (10/144)*rx2*(u1[midp,j,k-1]-2*u1[mid,j,k-1]+u1[midm,j,k-1])+\
                    (1/144)*rx2*(u1[midp,j-1,k+1]-2*u1[mid,j-1,k+1]+u1[midm,j-1,k+1])+\
                    (10/144)*rx2*(u1[midp,j-1,k]-2*u1[mid,j-1,k]+u1[midm,j-1,k])+\
                    (1/144)*rx2*(u1[midp,j-1,k-1]-2*u1[mid,j-1,k-1]+u1[midm,j-1,k-1])
                #
                A3=(1/144)*ry2*(u1[midp,j+1,k+1]-2*u1[midp,j,k+1]+u1[midp,j-1,k+1])+\
                    (10/144)*ry2*(u1[midp,j+1,k]-2*u1[midp,j,k]+u1[midp,j-1,k])+\
                    (1/144)*ry2*(u1[midp,j+1,k-1]-2*u1[midp,j,k-1]+u1[midp,j-1,k-1])+\
                    (10/144)*ry2*(u1[mid,j+1,k+1]-2*u1[mid,j,k+1]+u1[mid,j-1,k+1])+\
                    (100/144)*ry2*(u1[mid,j+1,k]-2*u1[mid,j,k]+u1[mid,j-1,k])+\
                    (10/144)*ry2*(u1[mid,j+1,k-1]-2*u1[mid,j,k-1]+u1[mid,j-1,k-1])+\
                    (1/144)*ry2*(u1[midm,j+1,k+1]-2*u1[midm,j,k+1]+u1[midm,j-1,k+1])+\
                    (10/144)*ry2*(u1[midm,j+1,k]-2*u1[midm,j,k]+u1[midm,j-1,k])+\
                    (1/144)*ry2*(u1[midm,j+1,k-1]-2*u1[midm,j,k-1]+u1[midm,j-1,k-1])
                #
                A4=(1/144)*rz2*(u1[midp,j+1,k+1]-2*u1[midp,j+1,k]+u1[midp,j+1,k-1])+\
                    (10/144)*rz2*(u1[midp,j,k+1]-2*u1[midp,j,k]+u1[midp,j,k-1])+\
                    (1/144)*rz2*(u1[midp,j-1,k+1]-2*u1[midp,j-1,k]+u1[midp,j-1,k-1])+\
                    (10/144)*rz2*(u1[mid,j+1,k+1]-2*u1[mid,j+1,k]+u1[mid,j+1,k-1])+\
                    (100/144)*rz2*(u1[mid,j,k+1]-2*u1[mid,j,k]+u1[mid,j,k-1])+\
                    (10/144)*rz2*(u1[mid,j-1,k+1]-2*u1[mid,j-1,k]+u1[mid,j-1,k-1])+\
                    (1/144)*rz2*(u1[midm,j+1,k+1]-2*u1[midm,j+1,k]+u1[midm,j+1,k-1])+\
                    (10/144)*rz2*(u1[midm,j,k+1]-2*u1[midm,j,k]+u1[midm,j,k-1])+\
                    (1/144)*rz2*(u1[midm,j-1,k+1]-2*u1[midm,j-1,k]+u1[midm,j-1,k-1])
                #
                S=(1/20736)*(F[midp,j+1,k+1,n+1]+10*F[midp,j+1,k+1,n]+F[midp,j+1,k+1,n-1])+\
                    (10/20736)*(F[midp,j+1,k,n+1]+10*F[midp,j+1,k,n]+F[midp,j+1,k,n-1])+\
                    (1/20736)*(F[midp,j+1,k-1,n+1]+10*F[midp,j+1,k-1,n]+F[midp,j+1,k-1,n-1])+\
                    (10/20736)*(F[midp,j,k+1,n+1]+10*F[midp,j,k+1,n]+F[midp,j,k+1,n-1])+\
                    (100/20736)*(F[midp,j,k,n+1]+10*F[midp,j,k,n]+F[midp,j,k,n-1])+\
                    (10/20736)*(F[midp,j,k-1,n+1]+10*F[midp,j,k-1,n]+F[midp,j,k-1,n-1])+\
                    (1/20736)*(F[midp,j-1,k+1,n+1]+10*F[midp,j-1,k+1,n]+F[midp,j-1,k+1,n-1])+\
                    (10/20736)*(F[midp,j-1,k,n+1]+10*F[midp,j-1,k,n]+F[midp,j-1,k,n-1])+\
                    (1/20736)*(F[midp,j-1,k-1,n+1]+10*F[midp,j-1,k-1,n]+F[midp,j-1,k-1,n-1])+\
                    (10/20736)*(F[mid,j+1,k+1,n+1]+10*F[mid,j+1,k+1,n]+F[mid,j+1,k+1,n-1])+\
                    (100/20736)*(F[mid,j+1,k,n+1]+10*F[mid,j+1,k,n]+F[mid,j+1,k,n-1])+\
                    (10/20736)*(F[mid,j+1,k-1,n+1]+10*F[mid,j+1,k-1,n]+F[mid,j+1,k-1,n-1])+\
                    (100/20736)*(F[mid,j,k+1,n+1]+10*F[mid,j,k+1,n]+F[mid,j,k+1,n-1])+\
                    (1000/20736)*(F[mid,j,k,n+1]+10*F[mid,j,k,n]+F[mid,j,k,n-1])+\
                    (100/20736)*(F[mid,j,k-1,n+1]+10*F[mid,j,k-1,n]+F[mid,j,k-1,n-1])+\
                    (10/20736)*(F[mid,j-1,k+1,n+1]+10*F[mid,j-1,k+1,n]+F[mid,j-1,k+1,n-1])+\
                    (100/20736)*(F[mid,j-1,k,n+1]+10*F[mid,j-1,k,n]+F[mid,j-1,k,n-1])+\
                    (10/20736)*(F[mid,j-1,k-1,n+1]+10*F[mid,j-1,k-1,n]+F[mid,j-1,k-1,n-1])+\
                    (1/20736)*(F[midm,j+1,k+1,n+1]+10*F[midm,j+1,k+1,n]+F[midm,j+1,k+1,n-1])+\
                    (10/20736)*(F[midm,j+1,k,n+1]+10*F[midm,j+1,k,n]+F[midm,j+1,k,n-1])+\
                    (1/20736)*(F[midm,j+1,k-1,n+1]+10*F[midm,j+1,k-1,n]+F[midm,j+1,k-1,n-1])+\
                    (10/20736)*(F[midm,j,k+1,n+1]+10*F[midm,j,k+1,n]+F[midm,j,k+1,n-1])+\
                    (100/20736)*(F[midm,j,k,n+1]+10*F[midm,j,k,n]+F[midm,j,k,n-1])+\
                    (10/20736)*(F[midm,j,k-1,n+1]+10*F[midm,j,k-1,n]+F[midm,j,k-1,n-1])+\
                    (1/20736)*(F[midm,j-1,k+1,n+1]+10*F[midm,j-1,k+1,n]+F[midm,j-1,k+1,n-1])+\
                    (10/20736)*(F[midm,j-1,k,n+1]+10*F[midm,j-1,k,n]+F[midm,j-1,k,n-1])+\
                    (1/20736)*(F[midm,j-1,k-1,n+1]+10*F[midm,j-1,k-1,n]+F[midm,j-1,k-1,n-1])
                #
                Fjk=A2+A3+A4+(ht**2)*S
                del A2, A3, A4, S
                # X_{0jk}
                X_0jk=py*pz*(FA[j+1,k+1,n+1]+FA[j+1,k-1,n+1]+FA[j-1,k+1,n+1]+FA[j-1,k-1,n+1]+\
                    FA[j+1,k+1,n-1]+FA[j+1,k-1,n-1]+FA[j-1,k+1,n-1]+FA[j-1,k-1,n-1]+\
                    -2*(FA[j+1,k+1,n]+FA[j+1,k-1,n]+FA[j-1,k+1,n]+FA[j-1,k-1,n]))\
                    +(1-2*py)*pz*(FA[j,k+1,n+1]+FA[j,k-1,n+1]+FA[j,k+1,n-1]+FA[j,k-1,n-1]+\
                    -2*(FA[j,k+1,n]+FA[j,k-1,n]))\
                    +py*(1-2*pz)*(FA[j+1,k,n+1]+FA[j-1,k,n+1]+FA[j+1,k,n-1]+FA[j-1,k,n-1]+\
                    -2*(FA[j+1,k,n]+FA[j-1,k,n]))\
                    +(1-2*py)*(1-2*pz)*(FA[j,k,n+1]-2*FA[j,k,n]+FA[j,k,n-1])
                #  X_{Nxjk}
                X_Nxjk=py*pz*(FB[j+1,k+1,n+1]+FB[j+1,k-1,n+1]+FB[j-1,k+1,n+1]+FB[j-1,k-1,n+1]+\
                    FB[j+1,k+1,n-1]+FB[j+1,k-1,n-1]+FB[j-1,k+1,n-1]+FB[j-1,k-1,n-1]+\
                    -2*(FB[j+1,k+1,n]+FB[j+1,k-1,n]+FB[j-1,k+1,n]+FB[j-1,k-1,n]))\
                    +(1-2*py)*pz*(FB[j,k+1,n+1]+FB[j,k-1,n+1]+FB[j,k+1,n-1]+FB[j,k-1,n-1]+\
                    -2*(FB[j,k+1,n]+FB[j,k-1,n]))\
                    +py*(1-2*pz)*(FB[j+1,k,n+1]+FB[j-1,k,n+1]+FB[j+1,k,n-1]+FB[j-1,k,n-1]+\
                    -2*(FB[j+1,k,n]+FB[j-1,k,n]))\
                    +(1-2*py)*(1-2*pz)*(FB[j,k,n+1]-2*FB[j,k,n]+FB[j,k,n-1])
                # b_jk
                bjk=np.zeros(len(mid), dtype=float)
                bjk[0]=-px*X_0jk
                bjk[Nx-2]=-px*X_Nxjk
                # Y
                X[mid,j,k]=spsolve(Ajk,Fjk+bjk, use_umfpack=True)
                del bjk, X_0jk, X_Nxjk
        # Part 2: Finding the values of Y for current time step
        for k in mid:
            for i in mid:
                Y_i0k=pz*(FC[i,k-1,n+1]+FC[i,k+1,n+1]+FC[i,k-1,n-1]+FC[i,k+1,n-1]\
                -2.*(FC[i,k-1,n]+FC[i,k+1,n]))+(1-2*pz)*(FC[i,k,n+1]-2*FC[i,k,n]+FC[i,k,n-1])
                Y_iNyj=pz*(FD[i,k-1,n+1]+FD[i,k+1,n+1]+FD[i,k-1,n-1]+FD[i,k+1,n-1]+\
                -2.*(FD[i,k-1,n]+FD[i,k+1,n]))+(1-2*pz)*(FD[i,k,n+1]-2*FD[i,k,n]+FD[i,k,n-1])
                # b_ik
                bik=np.zeros(len(mid), dtype=float)
                bik[0]=-py*Y_i0k
                bik[Nx-2]=-py*Y_iNyj
                # Y
                Y[i,mid,k]=spsolve(Aik,X[i,mid,k]+bik, use_umfpack=True)
                del bik
        # Part 3: Finding the values of u**{n+1} for current time step
        for j in mid:
            for i in mid:
                bij=np.zeros(len(mid), dtype=float)
                bij[0]=-pz*(FE[i,j,n+1]-2*FE[i,j,n]+FE[i,j,n-1])
                bij[Nx-2]=-pz*(FF[i,j,n+1]-2*FF[i,j,n]+FF[i,j,n-1])
                Z[i,j,mid]=spsolve(Aij,Y[i,j,mid]+bij, use_umfpack=True)
        u=Z+2*u1-u0
        u[0,:,:]=FA[:,:,n+1]
        u[Nx,:,:]=FB[:,:,n+1]
        u[:,0,:]=FC[:,:,n+1]
        u[:,Ny,:]=FD[:,:,n+1]
        u[:,:,0]=FE[:,:,n+1]
        u[:,:,Nz]=FF[:,:,n+1]
        u0=u1
        u1=u
    # Norm
    Norm=np.max(np.abs(u-Uexact))
    return Norm,u,Uexact

The results of analyzing the numerical solution of the problem, using the explicit scheme, is available at the following tables.

In [91]:
it=3
Norm=np.zeros(it, dtype=float)
N0=5;
Nt0=100;
N=np.asarray([N0*2**n for n in range(0,it)])
Nt=np.asarray([Nt0 for n in range(0,it)])
Ratio=np.zeros(it, dtype=float)
LOG=np.zeros(it, dtype=float)
# iteration
for n in range(it):
    Norm[n],_,_ =Example02_4thSolver3D(N[n],Nt[n])
    if (n>0):
        Ratio[n]=Norm[n-1]/Norm[n]
        LOG[n]=math.log(Ratio[n],2)
        
data = pd.DataFrame({'Nx': N, 'Ny': N, 'Nz': N, 'Nt': Nt, 'Norm': Norm, 'Ratio': Ratio, 'Log': LOG})
data

,Nx,Ny,Nz,Nt,Norm,Ratio,Log
0,5,5,5,100,1.940344e-05,0.000000,0.000000
1,10,10,10,100,1.246048e-06,15.571977,3.960880
2,20,20,20,100,7.809125e-08,15.956313,3.996055


In [92]:
it=3
Norm=np.zeros(it, dtype=float)
N0=10;
Nt0=20;
N=np.asarray([N0*2**n for n in range(0,it)])
Nt=np.asarray([Nt0*2**n for n in range(0,it)])
Ratio=np.zeros(it, dtype=float)
LOG=np.zeros(it, dtype=float)
# iteration
for n in range(it):
    Norm[n],_,_ =Example02_4thSolver3D(N[n],Nt[n])
    if (n>0):
        Ratio[n]=Norm[n-1]/Norm[n]
        LOG[n]=math.log(Ratio[n],2)
        
data = pd.DataFrame({'Nx': N, 'Ny': N, 'Nz': N, 'Nt': Nt, 'Norm': Norm, 'Ratio': Ratio, 'Log': LOG})
data

,Nx,Ny,Nz,Nt,Norm,Ratio,Log
0,10,10,10,20,1.247129e-06,0.000000,0.000000
1,20,20,20,40,8.111692e-08,15.374460,3.942464
2,40,40,40,80,5.068829e-09,16.003088,4.000278


What stands out from the tables is that the order of the method is 4 (in space and time). Moreover, consider the following experiment to test the stability condition.